## 2. Trial by trial decoding analysis

### (Map Ripple's decode, insert into the Database.)
[This notebook uses TrialChoice() and RippleTimes() to populate the TrialChoiceReplay() table.]
##### All the tables mentioned above are under shijiegu github/spyglass/shijiegu/Analysis_SGU.py. It is not in the Franklab spyglass.

Jun 23, 2024
Shijie Gu

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import spyglass as nd
import pandas as pd
import os
# ignore datajoint+jupyter async warnings
import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)
warnings.simplefilter('ignore', category=ResourceWarning)

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import logging
import multiprocessing

FORMAT = '%(asctime)s %(message)s'

logging.basicConfig(level='INFO', format=FORMAT, datefmt='%d-%b-%y %H:%M:%S')

from spyglass.common import (Session, IntervalList,IntervalPositionInfo,
                             LabMember, LabTeam, Raw, Session, Nwbfile,
                            Electrode,LFPBand,interval_list_intersect)
from spyglass.common.common_interval import _intersection

import spyglass.spikesorting as ss
from spyglass.spikesorting.v0 import (SortGroup, 
                                    SortInterval,
                                    SpikeSortingPreprocessingParameters,
                                    SpikeSortingRecording, 
                                    SpikeSorterParameters,
                                    SpikeSortingRecordingSelection,
                                    ArtifactDetectionParameters, ArtifactDetectionSelection,
                                    ArtifactRemovedIntervalList, ArtifactDetection,
                                      SpikeSortingSelection, SpikeSorting,
                                   CuratedSpikeSortingSelection,CuratedSpikeSorting,Curation)
#from spyglass.decoding.clusterless import (UnitMarks,
#                                           UnitMarkParameters,UnitMarksIndicatorSelection,
#                                          UnitMarksIndicator)
from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename
from spyglass.common.common_position import IntervalPositionInfo, IntervalPositionInfoSelection
from spyglass.common.common_position import IntervalLinearizedPosition

from spyglass.shijiegu.load import load_epoch_data
# Here are the analysis tables specific to Shijie Gu
from spyglass.shijiegu.Analysis_SGU import (TrialChoice,
                                   TrialChoiceReplay,
                                   RippleTimes,
                                   DecodeResultsLinear,get_linearization_map,
                                   find_ripple_times,mua_thresholder,sort_replays,
                                   classify_ripples,classify_ripple_content,
                                   load_everything,MUA,
                                   TetrodeNumber
                                  )
from spyglass.shijiegu.Analysis_SGU import EpochPos,RippleTimesWithDecode,ExtendedRippleTimesWithDecode,ExtendedRippleTimes,ExtendedTrialChoiceReplay,TrialChoiceReplayTransition
from spyglass.shijiegu.ripple_add_replay import (replay_parser_master,plot_ripples_replays)

[2025-04-11 16:43:22,442][INFO]: Connecting shijiegu-alt@lmf-db.cin.ucsf.edu:3306
11-Apr-25 16:43:22 Connecting shijiegu-alt@lmf-db.cin.ucsf.edu:3306
[2025-04-11 16:43:22,513][INFO]: Connected shijiegu-alt@lmf-db.cin.ucsf.edu:3306
11-Apr-25 16:43:22 Connected shijiegu-alt@lmf-db.cin.ucsf.edu:3306


In [15]:
# the only cell to be edited
nwb_file_name = 'molly20220419.nwb'

In [16]:
nwb_copy_file_name = get_nwb_copy_filename(nwb_file_name)

In [17]:
#((TetrodeNumber() & {'nwb_file_name': nwb_copy_file_name})).fetch("ca1_tetrode_ind")[0]

In [18]:
(RippleTimesWithDecode & {'nwb_file_name':nwb_copy_file_name})#.delete()

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,classifier_param_name a name for this set of parameters,encoding_set a name for this set of encoding,decode_threshold_method a name for this thresholding method,ripple_times ripple times within that interval
molly20220419_.nwb,02_Seq2Session1,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_M0SD,=BLOB=
molly20220419_.nwb,04_Seq2Session2,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_M0SD,=BLOB=
molly20220419_.nwb,06_Seq2Session3,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_M0SD,=BLOB=
molly20220419_.nwb,08_Seq2Session4,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_M0SD,=BLOB=
molly20220419_.nwb,10_Seq2Session5,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_M0SD,=BLOB=


In [7]:
(ExtendedRippleTimesWithDecode & {'nwb_file_name':nwb_copy_file_name})#.delete()

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,classifier_param_name a name for this set of parameters,encoding_set a name for this set of encoding,decode_threshold_method a name for this thresholding method,ripple_times ripple times within that interval


In [8]:
DecodeResultsLinear & {'nwb_file_name': nwb_copy_file_name}

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,classifier_param_name a name for this set of parameters,encoding_set a name for this set of encoding,posterior posterior within that interval (1D)
molly20220419_.nwb,02_Seq2Session1,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=
molly20220419_.nwb,04_Seq2Session2,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=
molly20220419_.nwb,06_Seq2Session3,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=
molly20220419_.nwb,08_Seq2Session4,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=
molly20220419_.nwb,10_Seq2Session5,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=


In [66]:
TrialChoice() & {'nwb_file_name':nwb_copy_file_name}

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),"epoch_name session name, get from IntervalList","choice_reward pandas dataframe, choice"
eliot20221017_.nwb,2,02_Seq2Session1,=BLOB=
eliot20221017_.nwb,4,04_Seq2Session2,=BLOB=
eliot20221017_.nwb,6,06_Seq2Session3,=BLOB=
eliot20221017_.nwb,8,08_Seq2Session4,=BLOB=


In [12]:
epoch_num = 2

### Begin testing

In [248]:
(RippleTimesWithDecode & {'nwb_file_name':nwb_copy_file_name})

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,classifier_param_name a name for this set of parameters,encoding_set a name for this set of encoding,decode_threshold_method a name for this thresholding method,ripple_times ripple times within that interval
lewis20240118_.nwb,02_Rev1Session1,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_0SD,=BLOB=
lewis20240118_.nwb,02_Rev1Session1,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_M05SD,=BLOB=
lewis20240118_.nwb,04_Rev1Session2,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_0SD,=BLOB=
lewis20240118_.nwb,04_Rev1Session2,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_M05SD,=BLOB=
lewis20240118_.nwb,06_Rev1Session3,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_0SD,=BLOB=
lewis20240118_.nwb,06_Rev1Session3,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_M05SD,=BLOB=
lewis20240118_.nwb,08_Rev1Session4,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_0SD,=BLOB=
lewis20240118_.nwb,08_Rev1Session4,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_M05SD,=BLOB=


In [26]:
log_df = pd.DataFrame((TrialChoice & {"nwb_file_name":"lewis20240118_.nwb",
               "epoch_name":"06_Rev1Session3"}).fetch1("choice_reward"))

In [27]:
ripple_times_ = pd.DataFrame((RippleTimesWithDecode & {'nwb_file_name':nwb_copy_file_name,
                         "decode_threshold_method":"MUA_M05SD",
                          "interval_list_name":"06_Rev1Session3"}).fetch1("ripple_times"))

In [30]:
log_df.loc[:40]

,timestamp_H,Home,timestamp_O,OuterWellIndex,rewardNum,current,future_H,future_O,past,past_reward
1,1.705608e+09,1.0,1.705608e+09,1.0,1.0,1.0,1.0,3.0,NaN,NaN
2,1.705608e+09,1.0,1.705608e+09,3.0,2.0,3.0,3.0,2.0,1.0,NaN
3,1.705608e+09,1.0,1.705608e+09,2.0,1.0,2.0,2.0,4.0,3.0,3.0
4,1.705608e+09,1.0,1.705608e+09,4.0,1.0,4.0,4.0,1.0,2.0,3.0
5,1.705608e+09,1.0,1.705608e+09,1.0,2.0,1.0,1.0,2.0,4.0,3.0
6,NaN,NaN,1.705608e+09,2.0,0.0,2.0,2.0,4.0,1.0,1.0
7,1.705608e+09,1.0,1.705608e+09,4.0,2.0,4.0,4.0,3.0,2.0,1.0
8,1.705608e+09,1.0,1.705608e+09,3.0,1.0,3.0,3.0,1.0,4.0,4.0
9,1.705608e+09,1.0,1.705608e+09,1.0,1.0,1.0,1.0,2.0,3.0,4.0
10,1.705608e+09,1.0,1.705608e+09,2.0,2.0,2.0,2.0,3.0,1.0,4.0


In [38]:
trial = 36
print(log_df.loc[trial].timestamp_H,log_df.loc[trial].timestamp_O)

nan 1705608820.8491178


In [39]:
trial = 37
print(log_df.loc[trial].timestamp_H,log_df.loc[trial].timestamp_O)

1705608863.3463516 1705608876.689425


In [37]:
ripple_times_.loc[160:200]

,event_number,start_time,animal_location,trial_number,cont_intvl,frag_intvl,cont_intvl_replay,end_time,duration,mean_zscore,median_zscore,max_zscore,min_zscore,speed_at_start,speed_at_end,max_speed,min_speed,median_speed,mean_speed
160,161,1.705609e+09,home,34,"[[1705608800.56899, 1705608800.6009898]]",[],[[0]],1.705609e+09,0.035000,3.901003,3.605126,5.772206,1.357145,2.315660,3.992457,3.992457,2.315660,3.111360,3.119185
161,162,1.705609e+09,well4,36,"[[1705608821.4829743, 1705608821.5629742]]",[],[[4]],1.705609e+09,0.083000,1.553521,0.971338,6.589257,-0.554812,0.933037,1.038826,1.070657,0.933037,1.040189,1.025278
162,163,1.705609e+09,well4,36,"[[1705608823.9369724, 1705608823.9869723]]",[],[[4]],1.705609e+09,0.053000,1.679122,0.626003,5.252738,0.048138,3.917924,2.312353,3.917924,1.885117,2.536417,2.623276
163,164,1.705609e+09,well4,36,"[[1705608825.596971, 1705608825.730971]]",[],[[4]],1.705609e+09,0.136001,1.498026,1.066184,4.469508,0.060136,0.490955,3.970643,3.970643,0.490955,1.750524,1.900098
164,165,1.705609e+09,well4,36,"[[1705608826.8169703, 1705608826.8809702]]",[],[[2]],1.705609e+09,0.065001,0.921683,0.610122,2.533761,-0.361819,1.080203,1.253109,1.253109,1.080203,1.166392,1.166011
165,166,1.705609e+09,well4,36,"[[1705608830.090968, 1705608830.2029676]]",[],[[4]],1.705609e+09,0.114001,1.185642,0.943075,3.135475,0.016352,0.379410,0.367254,0.379410,0.174309,0.272632,0.274088
166,167,1.705609e+09,well4,36,"[[1705608832.9729657, 1705608833.0669656]]",[],[[4]],1.705609e+09,0.095001,1.385889,1.229356,4.313527,-0.489542,2.331630,2.620628,2.620628,2.331630,2.520012,2.505744
167,168,1.705609e+09,well4,36,"[[1705608837.4609623, 1705608837.552962]]",[],[[4]],1.705609e+09,0.095001,1.736267,1.088151,5.854516,-0.551584,0.949458,2.659409,2.659409,0.949458,1.522226,1.604174
168,169,1.705609e+09,arm4,36,"[[1705608847.552955, 1705608847.8109546]]",[],"[[4, 0]]",1.705609e+09,0.259001,2.782327,1.553401,13.299902,-0.498721,0.556499,0.385159,0.556499,0.052218,0.326677,0.295954
169,170,1.705609e+09,arm4,36,"[[1705608848.0629542, 1705608848.1149542]]",[],[[4]],1.705609e+09,0.053000,1.316874,1.133353,2.846096,0.040155,0.061279,0.242689,0.242689,0.058299,0.153762,0.152871


In [205]:
ripple_times_.loc[2].start_time

1704827604.1483257

In [206]:
ripple_times_.loc[2].end_time

1704827604.5343268

In [209]:
log_df = pd.DataFrame((TrialChoice() & {'nwb_file_name':nwb_copy_file_name,
                  "epoch_name":"02_Rev2Session1"}).fetch1("choice_reward"))

In [213]:
log_df

,timestamp_H,Home,timestamp_O,OuterWellIndex,rewardNum,current,future_H,future_O,past,past_reward
1,1.704828e+09,1.0,1.704828e+09,1.0,2.0,1.0,1.0,3.0,NaN,NaN
2,1.704828e+09,1.0,1.704828e+09,3.0,1.0,3.0,3.0,4.0,1.0,1.0
3,1.704828e+09,1.0,1.704828e+09,4.0,1.0,4.0,4.0,2.0,3.0,1.0
4,1.704828e+09,1.0,1.704828e+09,2.0,2.0,2.0,2.0,1.0,4.0,1.0
5,1.704828e+09,1.0,1.704828e+09,1.0,1.0,1.0,1.0,3.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
78,1.704830e+09,1.0,1.704830e+09,1.0,2.0,1.0,1.0,2.0,3.0,3.0
79,1.704830e+09,1.0,1.704830e+09,2.0,2.0,2.0,2.0,4.0,1.0,1.0
80,1.704830e+09,1.0,1.704830e+09,4.0,2.0,4.0,4.0,3.0,2.0,2.0
81,1.704830e+09,1.0,1.704830e+09,3.0,2.0,3.0,3.0,NaN,4.0,4.0


In [211]:
log_df.loc[2].timestamp_H

1704827601.0818493

In [212]:
log_df.loc[2].timestamp_O

1704827610.7888799

### End testing

In [251]:
epoch_num = 2

In [ ]:
(epoch_name,log_df,ripple_times,decode,
     head_speed,head_orientation,linear_position_df,
     lfp_df,theta_df,ripple_df,neural_df,mua_df) = replay_parser_master(nwb_file_name,epoch_num,
                                                                        decode_threshold_method = 'MUA_M05SD',
                                                                       encoding_set = '2Dheadspeed_above_4')

[09:39:26][WARNING] Spyglass: Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.
29-Oct-24 09:39:26 Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.


epoch name 02_Rev1Session1
epoch name 02_Rev1Session1
epoch_pos_name pos 1 valid times


[2024-10-29 09:39:27,192][WARNING]: Skipped checksum for file with hash: 942f52b0-a55f-b4dd-81cd-a9d9e0712783, and path: /stelmo/nwb/analysis/lewis20240118/lewis20240118_AK43PFM89I.nwb
29-Oct-24 09:39:27 Skipped checksum for file with hash: 942f52b0-a55f-b4dd-81cd-a9d9e0712783, and path: /stelmo/nwb/analysis/lewis20240118/lewis20240118_AK43PFM89I.nwb


In [29]:
log_df

,timestamp_H,Home,timestamp_O,OuterWellIndex,rewardNum,ripple_H,ripple_O,replay_H,replay_O,ripple_ID_H,ripple_ID_O,current,future_H,future_O,past,past_reward
1,1.704665e+09,1.0,1.704665e+09,1.0,2.0,[],"[[[1704665025.3625145, 1704665025.5325146]]]",[],[[[1]]],[],[0],1.0,1.0,3.0,NaN,NaN
2,1.704665e+09,1.0,1.704665e+09,3.0,1.0,[],[],[],[],[],[],3.0,3.0,2.0,1.0,1.0
3,1.704665e+09,1.0,1.704665e+09,2.0,2.0,[],[],[],[],[],[],2.0,2.0,4.0,3.0,1.0
4,1.704665e+09,1.0,1.704665e+09,4.0,2.0,[],[],[],[],[],[],4.0,4.0,1.0,2.0,2.0
5,1.704665e+09,1.0,1.704665e+09,1.0,1.0,"[[[1704665107.002473, 1704665107.106473], [170...",[],"[[[], []]]",[],[1],[],1.0,1.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1.704667e+09,1.0,1.704667e+09,4.0,2.0,"[[[1704666606.3097098, 1704666606.4297097], [1...",[],"[[[], [], []]]",[],[73],[],4.0,4.0,1.0,3.0,2.0
79,1.704667e+09,1.0,1.704667e+09,1.0,1.0,[],[],[],[],[],[],1.0,1.0,2.0,4.0,4.0
80,1.704667e+09,1.0,1.704667e+09,2.0,1.0,"[[[1704666645.6816895, 1704666645.7816896]]]",[],[[[]]],[],[74],[],2.0,2.0,3.0,1.0,4.0
81,1.704667e+09,1.0,1.704667e+09,3.0,2.0,[],[],[],[],[],[],3.0,3.0,NaN,2.0,4.0


### 2. Visual Inspection

In [26]:
mua_thresh = (MUA & {'nwb_file_name' : nwb_copy_file_name,'interval_list_name':epoch_name}).fetch1('mean')

## find tetrodes with signal
groups_with_cell=(SpikeSortingRecordingSelection & {
        'nwb_file_name' : nwb_copy_file_name}).fetch('sort_group_id')
groups_with_cell=np.setdiff1d(groups_with_cell,[100,101])
channel_IDs = list(neural_df.keys())
    
spikeColInd = {}
for g in groups_with_cell:
    spikeColInd_ = np.argwhere(np.isin(channel_IDs,(Electrode() &  {'nwb_file_name' : nwb_copy_file_name,
                                                     'electrode_group_name':str(g)}).fetch('electrode_id'))).ravel()
    spikeColInd[g] = spikeColInd_

# Pre-make result directory:
decoding_path=(DecodeResultsLinear() & {'nwb_file_name':nwb_copy_file_name}).fetch('posterior')[0]
exampledir=os.path.join(os.path.split(decoding_path)[0],nwb_file_name,'SWR',epoch_name)
isExist = os.path.exists(exampledir)
if not isExist:
    os.makedirs(exampledir)
savefolder = exampledir
print(savefolder)

for t in log_df.index[:-2]:
    # Home
    ripples = log_df.loc[t].ripple_H
    ripple_table_inds = log_df.loc[t].ripple_ID_H

    if not np.isnan(log_df.loc[t].timestamp_H):
        plottimes = [log_df.loc[t].timestamp_H,log_df.loc[t].timestamp_H + 6]
        savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name + '_whole_trial'
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                        linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                        neural_df,mua_df,head_speed,head_orientation,
                        savename,savefolder,wholeTrial = 1,plottimes=plottimes,mua_thresh = mua_thresh)
            
    savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name
    plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                    linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                    neural_df,mua_df,head_speed,head_orientation,
                    savename,savefolder,wholeTrial = 0,mua_thresh = mua_thresh)

/cumulus/shijie/recording_pilot/lewis/decoding/lewis20240107.nwb/SWR/04_Rev2Session2


### 3. for all sessions

In [10]:
all_epochs = list((TrialChoice() & {'nwb_file_name':nwb_copy_file_name}).fetch('epoch'))
print(nwb_copy_file_name)

molly20220419_.nwb


In [11]:
all_epochs

[2, 4, 6, 8, 10]

#### without plot

In [ ]:
encoding_set = '2Dheadspeed_above_4' #'2Dheadspeed_above_4'
all_epochs = list((TrialChoice() & {'nwb_file_name':nwb_copy_file_name}).fetch('epoch'))
for epoch_num in all_epochs:
#for epoch_num in [2,6,8,10]:
    (epoch_name,log_df,ripple_times,decode,
     head_speed,head_orientation,linear_position_df,
     lfp_df,theta_df,ripple_df,neural_df,mua_df) = replay_parser_master(nwb_file_name,epoch_num,
                                                                        encoding_set = encoding_set,
                                                                        decode_threshold_method = 'MUA_M05SD',
                                                                        extended = False)

[22:39:01][WARNING] Spyglass: Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.
11-Apr-25 22:39:01 Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.


epoch name 02_Seq2Session1
epoch name 02_Seq2Session1
epoch_pos_name pos 1 valid times


[2025-04-11 22:39:04,168][WARNING]: Skipped checksum for file with hash: db487cf4-2333-efe8-29ac-1076ba3bbf42, and path: /stelmo/nwb/analysis/molly20220419/molly20220419_DYIFV8Y8E8.nwb
11-Apr-25 22:39:04 Skipped checksum for file with hash: db487cf4-2333-efe8-29ac-1076ba3bbf42, and path: /stelmo/nwb/analysis/molly20220419/molly20220419_DYIFV8Y8E8.nwb


In [13]:
%debug

> /home/shijiegu/Documents/spyglass/src/spyglass/shijiegu/load.py(146)load_epoch_data()
    144     except:
    145         TetrodeInd = np.array(CA1TetrodeInd).tolist() + np.array(CCTetrodeInd).tolist()
--> 146     lfp_df = pd.DataFrame(data=np.array(lfp)[:,TetrodeInd], index=lfp_t)
    147     lfp_df.index.name='time'
    148     lfp_df=xr.Dataset.from_dataframe(lfp_df)



ipdb>  np.array(lfp).shape


(19778055, 41)


ipdb>  TetrodeInd


array([16, 20, 25, 28, 33, 38, 42, 45, 49, 53])


...

ipdb>  np.array(CA1TetrodeInd).tolist() + np.array(CCTetrodeInd).tolist()


[0, 4, 9, 12, 17, 22, 26, 29, 33, 37, 16]


ipdb>  type([1,2,3]) == list


True


ipdb>  exit


#### with plots

In [ ]:
%matplotlib agg
all_epochs=list((TrialChoice() & {'nwb_file_name':nwb_copy_file_name}).fetch('epoch'))
for epoch_num in all_epochs:
    (epoch_name,log_df,ripple_times,decode,
     head_speed,head_orientation,linear_position_df,
     lfp_df,theta_df,ripple_df,neural_df,mua_df) = replay_parser_master(nwb_file_name,epoch_num,
                                                                        encoding_set = '2Dheadspeed_above_4',
                                                                        extended = False)

    """
    REAL WORK ENDS HERE, THE REST IS JUST PLOTTING
    """

    ## find tetrodes with signal
    groups_with_cell=(SpikeSortingRecordingSelection & {
        'nwb_file_name' : nwb_copy_file_name}).fetch('sort_group_id')
    groups_with_cell=np.setdiff1d(groups_with_cell,[100,101])
    channel_IDs = list(neural_df.keys())
    
    spikeColInd = {}
    for g in groups_with_cell:
        spikeColInd_ = np.argwhere(np.isin(channel_IDs,(Electrode() &  {'nwb_file_name' : nwb_copy_file_name,
                                                     'electrode_group_name':str(g)}).fetch('electrode_id'))).ravel()
        spikeColInd[g] = spikeColInd_

    # Pre-make result directory:
    decoding_path=(DecodeResultsLinear() & {'nwb_file_name':nwb_copy_file_name}).fetch('posterior')[0]
    exampledir=os.path.join(os.path.split(decoding_path)[0],nwb_file_name,'SWR',epoch_name)
    isExist = os.path.exists(exampledir)
    if not isExist:
        os.makedirs(exampledir)
    savefolder = exampledir
    print(savefolder)

    mua_thresh = (MUA & {'nwb_file_name' : nwb_copy_file_name,'interval_list_name':epoch_name}).fetch1('mean')


    for t in log_df.index[:-2]:
        # Home
        ripples = log_df.loc[t].ripple_H
        ripple_table_inds = log_df.loc[t].ripple_ID_H

        if not np.isnan(log_df.loc[t].timestamp_H):
            plottimes = [log_df.loc[t].timestamp_H,log_df.loc[t].timestamp_H + 6]
            savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name + '_whole_trial'
            plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                         linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                         neural_df,mua_df,head_speed,head_orientation,
                         savename,savefolder,wholeTrial = 1,plottimes=plottimes,mua_thresh = mua_thresh)
            
        savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                     linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                     neural_df,mua_df,head_speed,head_orientation,
                     savename,savefolder,wholeTrial = 0,mua_thresh = mua_thresh)
    
        
            
        # Outer Well
            
        ripples = log_df.loc[t].ripple_O
        ripple_table_inds = log_df.loc[t].ripple_ID_O

        plottimes = [log_df.loc[t].timestamp_O,log_df.loc[t].timestamp_O + 6]
        savename = 'trial' + str(t) + '_well_' + nwb_copy_file_name[:-5] + '_' + epoch_name + '_whole_trial'
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                     linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                     neural_df,mua_df,head_speed,head_orientation,
                     savename,savefolder,wholeTrial = 1,plottimes=plottimes,mua_thresh = mua_thresh)
        
        savename = 'trial' + str(t) + '_well_' + nwb_copy_file_name[:-5] + '_' + epoch_name
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                     linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                     neural_df,mua_df,head_speed,head_orientation,
                     savename,savefolder,wholeTrial = 0,mua_thresh = mua_thresh)
    
        

[15:56:46][WARNING] Spyglass: Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.
28-Oct-24 15:56:46 Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.


epoch name 02_Rev1Session1
epoch name 02_Rev1Session1
epoch_pos_name pos 1 valid times


[2024-10-28 15:56:48,245][WARNING]: Skipped checksum for file with hash: 942f52b0-a55f-b4dd-81cd-a9d9e0712783, and path: /stelmo/nwb/analysis/lewis20240118/lewis20240118_AK43PFM89I.nwb
28-Oct-24 15:56:48 Skipped checksum for file with hash: 942f52b0-a55f-b4dd-81cd-a9d9e0712783, and path: /stelmo/nwb/analysis/lewis20240118/lewis20240118_AK43PFM89I.nwb


In [135]:
((TetrodeNumber() & {'nwb_file_name': nwb_copy_file_name,"epoch_name":"02_Rev2Session1"})).fetch1("ca1_tetrode_ind")

array([  0,  14,  26,  46,  54,  66,  70,  95, 103, 107, 119, 124])

## Ends here

In [ ]:
"""
IMPORTANT!!!! RUN THIS FUNCTION EVERY TIME AFTER A NEW SESSION
USE OF GLOBAL VAR
"""

## find tetrodes with signal
groups_with_cell=(SpikeSortingRecordingSelection & {
    'nwb_file_name' : nwb_copy_file_name}).fetch('sort_group_id')
groups_with_cell=np.setdiff1d(groups_with_cell,[100,101])
channel_IDs = list(neural_df.keys())

spikeColInd = {}
for g in groups_with_cell:
    spikeColInd_ = np.argwhere(np.isin(channel_IDs,(Electrode() &  {'nwb_file_name' : nwb_copy_file_name,
                                                 'electrode_group_name':str(g)}).fetch('electrode_id'))).ravel()
    spikeColInd[g] = spikeColInd_

In [ ]:
spikeColInd

In [ ]:
(epoch_name,log_df,decode,head_speed,head_orientation,linear_position_df,
        lfp_df,theta_df,ripple_df,neural_df,mua_df) = load_epoch_data(nwb_copy_file_name,epoch_num)
ripple_times = pd.DataFrame((RippleTimes() & {'nwb_file_name': nwb_copy_file_name,
                                              'interval_list_name': epoch_name}).fetch1('ripple_times'))

In [ ]:
"""
IMPORTANT!!!! RUN THIS FUNCTION EVERY TIME AFTER A NEW SESSION
USE OF GLOBAL VAR
"""

def plot_ripples(ripples,ripple_table_inds,save_name,wholeTrial = 1):
    
    ripples = [r for r in ripples if len(r)>0]
    for ripple_ind in range(len(ripples)):
        t0t1 = np.vstack(ripples) #all possible shadings, just np.vstack(ripples), remove empty segments
        ripple_table_ind = ripple_table_inds[ripple_ind]
        
        if wholeTrial:
            plottimes = [t0t1.ravel()[0]-1,t0t1.ravel()[-1]+1]
            title = f"{save_name} all ripples"
            savename = save_name
        else:
            savename = save_name + str(ripple_table_ind)
            current_t0t1 = ripples[ripple_ind]
            plottimes = [current_t0t1.ravel()[0]-1,current_t0t1.ravel()[-1]+1]
    
            duration = int(ripple_times.loc[ripple_table_ind].duration * 1000)
            max_zscore = round(ripple_times.loc[ripple_table_ind].max_zscore,2)
            title = f"{savename} \n duration: {duration} ms, max_zscore: {max_zscore}"
            title += '\n Decoded arms:' + str(ripple_times.loc[ripple_table_ind].cont_intvl_replay)
    
        plot_decode_spiking(plottimes,t0t1,linear_position_df,decode,lfp_df,theta_df,
              neural_df,mua_df,ripple_df,head_speed,head_orientation,
              ripple_consensus_trace=None,
              title=title,savefolder=savefolder,savename=savename,
              simple=True,tetrode2ind = spikeColInd)
        if wholeTrial:
            break
    

In [ ]:
# Pre-make result directory:
decoding_path=(Decode() & {'nwb_file_name':nwb_copy_file_name}).fetch('posterior')[0]
exampledir=os.path.join(os.path.split(decoding_path)[0],nwb_file_name,'SWR',epoch_name)
isExist = os.path.exists(exampledir)
if not isExist:
    os.makedirs(exampledir)
savefolder = exampledir
print(savefolder)

In [ ]:
for t in log_df.index[:-2]:

    # Home
    ripples = log_df.loc[t].ripple_H
    ripple_table_inds = log_df.loc[t].ripple_ID_H
    savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name
    plot_ripples(ripples,ripple_table_inds,savename,wholeTrial = 0)
    plot_ripples(ripples,ripple_table_inds,savename,wholeTrial = 1)
    
    # Outer Well
    
    ripples = log_df.loc[t].ripple_O
    ripple_table_inds = log_df.loc[t].ripple_ID_O
    savename = 'trial' + str(t) + '_well_' + nwb_copy_file_name[:-5] + '_' + epoch_name
    plot_ripples(ripples,ripple_table_inds,savename,wholeTrial = 0)
    plot_ripples(ripples,ripple_table_inds,savename,wholeTrial = 1)


In [ ]:
#plt.plot(theta_t,theta_d)

### Plotting

In [ ]:
%%capture stored_output
add_replay(2)
add_replay(4)
add_replay(6)
add_replay(8)

### for whole day

In [ ]:
all_epochs=list((TrialChoice() & {'nwb_file_name':nwb_copy_file_name}).fetch('epoch'))
all_epochs

In [ ]:
%%capture stored_output
with multiprocessing.Pool(3) as p:
    p.map(add_replay,all_epochs)

In [ ]:
stored_output.show()

In [ ]:
print('here')

In [ ]:
add_replay(8)

In [ ]:
%%capture stored_output
add_replay(2)
add_replay(4)
add_replay(6)
add_replay(8)
add_replay(10)

In [ ]:
add_replay(10)
add_replay(12)

In [ ]:
    epoch_num=8
    
    # 1. Load state script
    key={'nwb_file_name':nwb_copy_file_name,'epoch':epoch_num}
    log=(TrialChoice & key).fetch1('choice_reward')
    epoch_name=(TrialChoice & key).fetch1('epoch_name')
    epoch_pos_name='pos '+str(int(epoch_name[:2])-1)+' valid times'
    print('epoch name',epoch_name)
    log_df=pd.DataFrame(log)

In [ ]:
log_df.loc[66,:]

In [ ]:
log_df.loc[66,:].timestamp_H

In [ ]:
linear_position_df = (IntervalLinearizedPosition() &
                      {'nwb_file_name': nwb_copy_file_name,
                       'interval_list_name': epoch_pos_name,
                       'position_info_param_name': 'default_decoding'}
                         ).fetch1_dataframe()

In [ ]:
    t=68
    
    # for each trial, find important times
    t_home=log_df.loc[t,'timestamp_H']
    t_outer=log_df.loc[t,'timestamp_O']
    if t+1<len(log_df):
        t_end=log_df.loc[t+1,'timestamp_H']
    else:
        t_end=log_df.loc[t,'timestamp_H']+10
    
    ''' HOME '''
    # find times close to home well
    time_slice = slice(t_home,t_outer)
    current_location = node_location['home']
    position_subset=linear_position_df.loc[time_slice]
    close_ind=np.argwhere(np.abs(position_subset.linear_position.values-
                                 current_location)<5).ravel()

In [ ]:
position_subset.index[0]

In [ ]:
position_subset

In [ ]:
    epoch_num=6
    
    # 1. Load state script
    key={'nwb_file_name':nwb_copy_file_name,'epoch':epoch_num}
    log=(TrialChoice & key).fetch1('choice_reward')
    epoch_name=(TrialChoice & key).fetch1('epoch_name')
    epoch_pos_name='pos '+str(int(epoch_name[:2])-1)+' valid times'
    print('epoch name',epoch_name)
    log_df=pd.DataFrame(log)

    # 1.5 Pre-make result directory:
    decoding_path=(Decode() & {'nwb_file_name':nwb_copy_file_name}).fetch('posterior')[0]
    exampledir=os.path.join(os.path.split(decoding_path)[0],nwb_file_name,epoch_name)
    isExist = os.path.exists(exampledir)
    if not isExist:
        os.makedirs(exampledir)

    # 2. Pre-expand result: Augment choice_reward table to be choice_reward_replay
    log_df_replay=log_df.copy()
    log_df_replay.insert(5,'ripple_H',[[] for i in range(len(log_df))]) #hold ripple times
    log_df_replay.insert(6,'ripple_O',[[] for i in range(len(log_df))])
    log_df_replay.insert(7,'replay_H',[[] for i in range(len(log_df))]) #hold decoded replay
    log_df_replay.insert(8,'ripple_peak_H',[[] for i in range(len(log_df))]) #hold decoded replay
    log_df_replay.insert(9,'replay_O',[[] for i in range(len(log_df))])
    log_df_replay.insert(10,'ripple_peak_O',[[] for i in range(len(log_df))]) #hold decoded replay

    # 3. linear map
    linear_map,node_location=get_linearization_map()

    # 4. load position, decoding, ripple times, mua threshold
    (linear_position_df,decode,ripple_nwb,
     ripple_timestamps,ripple_times,
     recordings,neural_ts,
     mua,mua_time)=load_everything(nwb_copy_file_name,
                    epoch_name,
                    epoch_pos_name)
    


In [ ]:
(IntervalList & {'nwb_file_name': nwb_copy_file_name,'interval_list_name': epoch_pos_name}).fetch1('valid_times')[0]

In [ ]:
    position_info = (IntervalPositionInfo() & {'nwb_file_name': nwb_copy_file_name,
                     'interval_list_name': epoch_pos_name,
                     'position_info_param_name': 'default'}).fetch1_dataframe()

In [ ]:
log_df.loc[60].timestamp_H

In [ ]:
log_df.loc[61].timestamp_H

In [ ]:
log_df.loc[82].timestamp_H

In [ ]:
ripple_timestamps[0]

In [ ]:
position_info.index[0]

In [ ]:
plt.plot(linear_position_df.loc[linear_position_df.index>1650309121,'linear_position'])

In [ ]:
add_replay(10)

In [ ]:
TrialChoiceReplay() & {'nwb_file_name':nwb_copy_file_name}

In [ ]:
choice_reward_replay=(TrialChoiceReplay() & {'nwb_file_name':nwb_copy_file_name,
                                            'epoch':2}).fetch1('choice_reward_replay')

In [ ]:
pd.DataFrame(choice_reward_replay)

### one trial

In [ ]:
mua_threshold=np.mean(mua)
mua_threshold

### all trials

In [ ]:
numWorker=30
print('num of worker:',numWorker)
with multiprocessing.Pool(numWorker) as p:
    results=p.map(find_ripple_classify_decode,legal_trials)

In [ ]:
len(results)

In [ ]:
for i in range(len(legal_trials)):
    t=legal_trials[i]
    # add to the choice_reward_replay
    log_df_replay.at[t, 'ripple_H']=results[i][0]
    log_df_replay.at[t, 'ripple_O']=results[i][3]
    
    log_df_replay.at[t, 'replay_H']= results[i][1]
    log_df_replay.at[t, 'replay_O']= results[i][4]
    
    log_df_replay.at[t, 'ripple_peak_H']= results[i][2]
    log_df_replay.at[t, 'ripple_peak_O']= results[i][5]
log_df_replay_subset=log_df_replay.loc[legal_trials,:].copy()

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
log_df_replay

In [ ]:
# insert into Spyglass
key={'nwb_file_name':nwb_copy_file_name,'epoch':epoch_num,'choice_reward_replay':log_df_replay_subset.to_dict()}
TrialChoiceReplay().make(key,replace=True)

In [ ]:
TrialChoiceReplay()

## investigate short ripples

In [ ]:
short_ripple_H=[]
for i in range(len(legal_trials)):
    t=trial_ind[i]
    ripple_H=log_df_replay.loc[t, 'ripple_H']
    for ri in range(len(ripple_H)):
        if len(ripple_H[ri])==0: #state time is short
            continue
        ripple_=np.reshape(ripple_H[ri],(1,-1))
        if (ripple_[0,-1]-ripple_[0,0])<=0.05:
            short_ripple_H.append((t,ri+1))

In [ ]:
replay_ripple_count=np.zeros((10,5))
replay_ripple=np.zeros((10,5),dtype=np.object)
for i in range(len(trial_ind)):
    t=trial_ind[i]
    replay_all=log_df_replay.loc[t,'replay_O']
    ripple_all=log_df_replay.loc[t,'ripple_peak_O']
    
    for ri in range(len(replay_all)):
        replay=replay_all[ri]
        unique_arms=np.unique(np.setdiff1d(replay,[5]))
        replay_arm_num=len(unique_arms)
    
        ripple_num=len(ripple_all[ri])
        if ripple_num<=10 and replay_arm_num<=5:
            replay_ripple_count[ripple_num-1,replay_arm_num-1]+=1
            if type(replay_ripple[ripple_num-1,replay_arm_num-1])=='list':
                replay_ripple[ripple_num-1,replay_arm_num-1].append(replay)
            else:
                replay_ripple[ripple_num-1,replay_arm_num-1]=replay

In [ ]:
replay_ripple[:,2]

In [ ]:
fig, axes = plt.subplots(nrows=1)
plt.imshow(np.flipud(replay_ripple_count.T))
axes.set_xlabel('ripple peak number')
axes.set_ylabel('distinct arms in replay')
axes.set_yticks([2,3,4])
axes.set_yticklabels(['3','2','1'])